Credit Card Fraud Detection demo:

The notebook demonstrates how to conduct a fraud detection application with the BigDL deep learning library on Apache Spark. We'll try to introduce some techniques that can be used for training a fraud detection model, but some advanced skills is not applicable since the dataset is highly simplified.

**Dataset:**
Credit Card Fraud Detection
https://www.kaggle.com/dalpozz/creditcardfraud

This dataset presents transactions that occurred in two days, where we have 492 frauds out of 284,807 transactions. The dataset is highly unbalanced, the positive class (frauds) account for 0.172% of all transactions.

It contains only numerical input variables which are the result of a PCA transformation. Unfortunately, due to confidentiality issues, we cannot provide the original features and more background information about the data. Features V1, V2, ... V28 are the principal components obtained with PCA, the only features which have not been transformed with PCA are 'Time' and 'Amount'. Feature 'Time' contains the seconds elapsed between each transaction and the first transaction in the dataset. The feature 'Amount' is the transaction Amount, this feature can be used for example-dependant cost-senstive learning. Feature 'Class' is the response variable and it takes value 1 in case of fraud and 0 otherwise.

**Software stack:**
Scala 2.11 + Spark 2.1 + BigDL Master




Loading data from csv files and output the schema:

In [1]:
import org.apache.log4j.{Level, Logger}
import org.apache.spark.ml.Pipeline
import org.apache.spark.ml.linalg._
import org.apache.spark.ml.classification.{LogisticRegression, LogisticRegressionModel, MultilayerPerceptronClassifier, _}
import org.apache.spark.ml.evaluation.{BinaryClassificationEvaluator, MulticlassClassificationEvaluator}
import org.apache.spark.ml.feature.{MinMaxScaler, StandardScaler, VectorAssembler}
import org.apache.spark.ml.tuning.{ParamGridBuilder, TrainValidationSplit}
import org.apache.spark.mllib.evaluation.BinaryClassificationMetrics
import org.apache.spark.sql.{Row, SparkSession}
import org.apache.spark.sql.functions._

val conf = Engine.createSparkConf()
val spark = SparkSession.builder().master("local[1]").appName("ss").config(conf).getOrCreate()
import spark.implicits._


val raw = spark.read.format("csv").option("header", "true").option("mode", "DROPMALFORMED").csv("data/creditcard.csv")
val df = raw.select(((1 to 28).map(i => "V" + i) ++ Array("Time", "Amount", "Class")).map(s => col(s).cast("Double")): _*)

println("total: " + df.count())
df.printSchema()

total: 284807                                                                   
root
 |-- V1: double (nullable = true)
 |-- V2: double (nullable = true)
 |-- V3: double (nullable = true)
 |-- V4: double (nullable = true)
 |-- V5: double (nullable = true)
 |-- V6: double (nullable = true)
 |-- V7: double (nullable = true)
 |-- V8: double (nullable = true)
 |-- V9: double (nullable = true)
 |-- V10: double (nullable = true)
 |-- V11: double (nullable = true)
 |-- V12: double (nullable = true)
 |-- V13: double (nullable = true)
 |-- V14: double (nullable = true)
 |-- V15: double (nullable = true)
 |-- V16: double (nullable = true)
 |-- V17: double (nullable = true)
 |-- V18: double (nullable = true)
 |-- V19: double (nullable = true)
 |-- V20: double (nullable = true)
 |-- V21: double (nullable = true)
 |-- V22: double (nullable = true)
 |-- V23: double (nullable = true)
 |-- V24: double (nullable = true)
 |-- V25: double (nullable = true)
 |-- V26: double (nullable = true)
 |-- V27: dou

**Feature analysis:**

Normally it would improve the model if we could derive more features from the raw transaction records. E.g.
    days to last transaction,
    distance with last transaction,
    amount percentage over the last 1 month / 3months
    ...

Yet with the public dataset, we can hardly derive any extention features from the PCA result. So here we only introduce several general practices:
1. Usually there's a lot of categorical data in the raw dataset, E.g. post code, card type, merchandise id, seller id, etc.
    1). For categorical feature with limited candidate values, like card type, channel id, just use OneHotEncoder.
    2). For categorical feature with many candidate values, like merchandise id, post code or even phone number, suggest to use Weight of Evidence.
    3). You can also use FeatureHasher from Spark MLlib which will be release with Spark 2.3.


For this dataset, essentially it's only a classification training problem with highly unbalanced data set.

** Approach **
1. We will build a feature transform pipeline with Apache Spark and some of our transformers.
2. We will run some inital statistical analysis and split the dataset for training and validation.
3. We will build the model with BigDL.
4. We will compare different strategy to handle the unbalance.
4. We will evaluate the model with grid search and area under precision-recall curve.

Details of each step is as follows:


***step 1. Build an inital pipeline for feature transform.***

For each training records, we intend to aggregate all the features into one Spark Vector, which will then be sent to BigDL model for the training. First we'd like to introduce one handy transformer that we developed to help user build custom Transformers for Spark ML Pipeline. 
    
    
    ```
    class FuncTransformer (
      override val uid: String,
      val func: UserDefinedFunction
    ) extends Transformer with HasInputCol with HasOutputCol with DefaultParamsWritable {
    ```
`FuncTransformer` takes an udf as the constructor parameter and use the udf to perform the actual transform. The transformer can be saved/loaded as other transformer and can be integrated into a pipeline normally. It can be used widely in many use cases like conditional conversion(if...else...), , type conversion, to/from Array, to/from Vector and many string ops.
Some examples: 
`val labelConverter = new FuncTransformer(udf { i: Double => if (i >= 1) 1 else 0 })`
    
`val shifter = new FuncTransformer(udf { i: Double => i + 1 })`
    
`val toVector = new FuncTransformer(udf { i: Double => Vectors.dense(i) })`
    
We will use `VectorAssembler` to compose the all the Vx columns and append the Amount column. Then use `StandardScaler` to normlize the training records. Since in BigDL, the criterion generally only accepts 1, 2, 3... as the Label, so we will replace all the 0 with 2 in the training data.
    



In [9]:
import com.intel.analytics.bigdl.nn._
import com.intel.analytics.bigdl.tensor.TensorNumericMath.TensorNumeric.NumericFloat
import org.apache.spark.ml.feature.{FuncTransformer, MinMaxScaler, StandardScaler, VectorAssembler}
import org.apache.spark.sql.functions._

val labelConverter = new FuncTransformer(udf {d: Double => if (d==0) 2 else d }).setInputCol("Class").setOutputCol("Class")
val assembler = new VectorAssembler().setInputCols((1 to 28).map(i => "V" + i).toArray ++ Array("Amount")).setOutputCol("assembled")
val scaler = new StandardScaler().setInputCol("assembled").setOutputCol("features")
val pipeline = new Pipeline().setStages(Array(assembler, scaler, labelConverter))
val pipelineModel = pipeline.fit(df)
val data = pipelineModel.transform(df)
data.select("features", "Class").show()

[Stage 15:=======>                                                  (1 + 7) / 8]+--------------------+-----+
|            features|Class|
+--------------------+-----+
|[-0.6942411021638...|  2.0|
|[0.60849525943109...|  2.0|
|[-0.6934992452238...|  2.0|
|[-0.4933240320774...|  2.0|
|[-0.5913287255806...|  2.0|
|[-0.2174742415711...|  2.0|
|[0.62779408220828...|  2.0|
|[-0.3289277697329...|  2.0|
|[-0.4565722152677...|  2.0|
|[-0.1726974406947...|  2.0|
|[0.73980031932134...|  2.0|
|[0.19654824114227...|  2.0|
|[0.63817910856358...|  2.0|
|[0.54596205586179...|  2.0|
|[-1.4253641430361...|  2.0|
|[-0.3841418567777...|  2.0|
|[0.56323980125506...|  2.0|
|[-0.2230591756515...|  2.0|
|[-2.7575786155874...|  2.0|
|[0.76220920780347...|  2.0|
+--------------------+-----+
only showing top 20 rows



***step 2. split the dataset into training and validation dataset.***

Unlike some other training dataset, where the data does not have a time of occurance. For this case, we can know the sequence of the transactions from the Time column. Thus randomly splitting the data into training and validation does not make much sense, since in real world applications, we can only use the history transactions for training and use the latest transactions for validation. Thus we'll split the dataset according the time of occurance. 


In [11]:
    val splitTime = data.stat.approxQuantile("Time", Array(0.7), 0.001).head

    val trainingData = data.filter(s"Time<$splitTime").cache()
    val validData = data.filter(s"Time>=$splitTime").cache()
    println("training count: " + trainingData.count())
    println("validData count: " + validData.count())

training count: 199424                                                          
validData count: 85383                                                          


***step 3. Build the model with BigDL***

From the research community and industry feedback, a simple neural network turns out be the perfect candidate for the fraud detection training. We will quickly build a multiple layer Perceptron with linear layers.
```
    val bigDLModel = Sequential()
      .add(Linear(29, 10))
      .add(Linear(10, 2))
      .add(LogSoftMax())
    val criterion = ClassNLLCriterion()
      ```
BigDL provides `DLEstimator` and `DLClassifier` for users with Apache Spark MLlib experience, which
provides high level API for training a BigDL Model with the Apache Spark `Estimator`/`Transfomer`
pattern, thus users can conveniently fit BigDL into a ML pipeline. The fitted model `DLModel` and
`DLClassiferModel` contains the trained BigDL model and extends the Spark ML `Model` class.
Alternatively users may also construct a `DLModel` with a pre-trained BigDL model to use it in
Spark ML Pipeline for prediction.

`DLClassifier` is a specialized `DLEstimator` that simplifies the data format for
classification tasks. It only supports label column of DoubleType, and the fitted
`DLClassifierModel` will have the prediction column of DoubleType.

For this case we'll just use `DLClassifier` for the training. Note that users can set differet optimization mothod, batch size and epoch number.     
      


In [ ]:

import com.intel.analytics.bigdl.nn._
import com.intel.analytics.bigdl.tensor.TensorNumericMath.TensorNumeric.NumericFloat
import com.intel.analytics.bigdl.utils.Engine
import org.apache.log4j.{Level, Logger}
import org.apache.spark.ml.ensemble.Bagging
import org.apache.spark.ml.evaluation.{BinaryClassificationEvaluator, MulticlassClassificationEvaluator}
import org.apache.spark.ml.feature.{FuncTransformer, MinMaxScaler, StandardScaler, VectorAssembler}
import org.apache.spark.ml.{DLClassifier, Pipeline}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.{DataFrame, SparkSession}

val bigDLModel = Sequential().add(Linear(29, 10)).add(Linear(10, 2)).add(LogSoftMax())
val criterion = ClassNLLCriterion()
val dlClassifier = new DLClassifier(bigDLModel, criterion, Array(29)).setLabelCol("Class").setBatchSize(trainingData.count().toInt).setMaxEpoch(200)
val model = dlClassifier.fit(trainingData)

Model Evaluation:
Given the class imbalance ratio, we recommend measuring the accuracy using the Area Under the Precision-Recall Curve (AUPRC). Confusion matrix accuracy is not meaningful for unbalanced classification.

In [ ]:
    val labelConverter2 = new FuncTransformer(udf {d: Double => if (d==2) 0 else d }).setInputCol("Class").setOutputCol("Class")
    val labelConverter3 = new FuncTransformer(udf {d: Double => if (d==2) 0 else d }).setInputCol("prediction").setOutputCol("prediction")
    val finalData = labelConverter2.transform(labelConverter3.transform(model.transform(validData)))
    
    val metrics = new BinaryClassificationEvaluator().setRawPredictionCol("prediction").setLabelCol("Class")
    val auPRC = metrics.evaluate(prediction)
    println("Area under precision-recall curve = " + auPRC)
    
    val recall = new MulticlassClassificationEvaluator()
      .setLabelCol("Class")
      .setMetricName("weightedRecall")
      .evaluate(prediction)
    println("recall = " + recall)

    val precisoin = new MulticlassClassificationEvaluator()
      .setLabelCol("Class")
      .setMetricName("weightedPrecision")
      .evaluate(prediction)
    println("Precision = " + precisoin)


In [ ]:
Area under precision-recall curve = 0.75

recall = 0.9995456054806969
Precision = 0.9995236507066619

To this point, we have finished the training and evaluation with a simple BigDL model. Next we'll try to optimize the training process. 

***step 4. handle the data imbalance***

There are several ways to approach this classification problem taking into consideration this unbalance.
1. Collect more data? Nice strategy but not applicable in this case
2. Resampling the dataset
    Essentially this is a method that will process the data to have an approximate 50-50 ratio.
    One way to achieve this is by OVER-sampling, which is adding copies of the under-represented class (better when you have little data)
    Another is UNDER-sampling, which deletes instances from the over-represented class (better when he have lot's of data)
3. Apart from under and over sampling, there is a very popular approach called SMOTE (Synthetic Minority Over-Sampling Technique), which is a combination of oversampling and undersampling, but the oversampling approach is not by replicating minority class but constructing new minority class data instance via an algorithm.


We'll start with Resampling.

Since there're 492 frauds out of 284,807 transactions, to build a reasonable training dataset, we'll use UNDER-sampling for normal transactions and use OVER-sampling for fraud transactions. By using the sampling rate as 
fraud -> 10, normal -> 0.05, we can get a training dataset of (5K fraud + 14K normal) transactions. We can use the training data to fit a model.

Yet we'll soon find that since there're only 5% of all the normal transactions are included in the training data, the model can only cover 5% of all the normal transactions, which is obviousely not optimistic. So how can we get a better converage for the normal transactions without breaking the ideal ratio in the training dataset?

An immediate improvement would be to train multiple models. For each model, we will run the resampling from the original dataset and get a new training data set. After training, we can select best voting strategy for all the models to make the prediction.

We'll use Ensembling of neural networks. That's where a Bagging classifier comes handy. Bagging is an Estimator we developed for ensembling of multiple other Estimator.


In [ ]:
package org.apache.spark.ml.ensemble

class Bagging[M <: Model[M]](override val uid: String)
  extends Estimator[BaggingModel[M]]
  with BaggingParams[M] {
  
  

In [ ]:
For usage, user need to set the specific Estimator to use and the number of models to be trained:

In [ ]:
    val estimator = new Bagging()
      .setPredictor(dlClassifier)
      .setLabelCol("Class")
      .setIsClassifier(true)
      .setNumModels(10)

Internally, Bagging will train $(numModels) models. Each model is trained with the resampled data from the original dataset.

In [ ]:
    val models = (0 until $(numModels)).map { _ =>
      val sampler = new StratifiedSampler(Map(2 -> 0.05, 1-> 10, 0 -> 1)).setLabel("Class")
      val bootstrapSample = sampler.transform(dataset)
      val oldclassifier = $(predictor).asInstanceOf[DLClassifier[Float]]
      val dlClassifier = new DLClassifier(oldclassifier.model, oldclassifier.criterion, Array(29)).setLabelCol("Class")
        .setBatchSize(oldclassifier.getBatchSize)
        .setMaxEpoch(oldclassifier.getMaxEpoch)
      dlClassifier.fit(bootstrapSample).asInstanceOf[M]
    }

After fitting, we can tune the voting strategy via `model.setThreshold(t)`. If using Threshold = 5, we can get the improved model

In [ ]:

Area under precision-recall curve = 0.9295288238964008
[Stage 8428:============================>                           (1 + 1) / 2]recall = 0.9922262595206485
[Stage 8434:============================>                           (1 + 1) / 2]Precision = 0.998684143354997